In [1]:
%pip install Uq upgini catboost

     |████████████████████████████████| 79 kB 3.5 MB/s eta 0:00:011
     |████████████████████████████████| 22.0 MB 42.0 MB/s eta 0:00:01    |█▋                              | 1.1 MB 20.3 MB/s eta 0:00:02     |████▋                           | 3.2 MB 20.3 MB/s eta 0:00:01     |████████████▏                   | 8.4 MB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 246 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 777 kB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 24.3 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 3.2 MB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 269 kB 24.5 MB/s eta 0:00:01
  Created wheel for Uq: filename=uq-0.1.2-py3-none-any.whl size=9176 sha256=c3cc069f071b04ea9184f3639bf38be783d4f4a55b32feb825ae7f1db6d20923
  Stored in directory: /Users/Fahad/L

Prepare Input Data

In [1]:
from os.path import exists
import pandas as pd
df_path = "train.csv.zip" if exists ("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df=pd.read_csv(df_path)
df=df.sample(n=19_000, random_state=0)
df["store"] =df ["store"].astype(str)
df["item"] =df ["item"].astype(str)
df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace = True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [2]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [3]:
train_feature = train.drop(columns = ["sales"])
train_target = train["sales"]
test_features = test.drop(columns = ["sales"])
test_target = test["sales"]

Enrich Features

In [4]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv=CVType.time_series)
enricher.fit(train_feature, train_target, eval_set =[(test_features, test_target)])

Detected task type: ModelTaskType.REGRESSION


Column name,Status,Description
date,All valid,All values in this column are good to go
target,All valid,All values in this column are good to go


Running search request with search_id=aa0336f6-483e-4e0a-af3e-b9b440f50f1c
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


26 relevant feature(s) found with the search keys: ['date'].


/opt/anaconda3/lib/python3.9/site-packages/upgini/features_enricher.py:1363: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(self.features_info.head(60).style.hide_index())


provider,source,feature name,shap value,coverage %,type,feature type
,,item,0.488457,100.000000,categorical,
,,store,0.172590,100.000000,categorical,
Upgini,Community shared,f_weather_date_weather_pca_0_d7e0a1fc,0.056901,100.000000,numerical,Free
Upgini,Community shared,f_events_date_week_sin1_847b5db1,0.047424,100.000000,numerical,Free
Upgini,Community shared,f_events_date_week_cos1_f6a8c1fc,0.029408,100.000000,numerical,Free
Upgini,Community shared,f_weather_date_weather_umap_48_b39cd0c4,0.025571,100.000000,numerical,Free
Upgini,Community shared,f_weather_date_weather_umap_24_2e14c9a6,0.018662,100.000000,numerical,Free
Upgini,Community shared,f_weather_date_weather_umap_33_89bb7578,0.015400,100.000000,numerical,Free
Upgini,Community shared,f_events_date_year_cos1_9014a856,0.012970,100.000000,numerical,Free
Upgini,Community shared,f_financial_date_silver_14e835ea,0.006757,100.000000,numerical,Free


In [8]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(train_feature, train_target, eval_set=[(test_features, test_target)], estimator = model, scoring = "mean_absolute_percentage_error")

Calculating metrics...
Done


,match_rate,baseline mean_absolute_percentage_error,enriched mean_absolute_percentage_error,uplift
,,,,
train,100.0,0.254322,0.166744,0.087578
eval 1,100.0,0.267351,0.187815,0.079536


In [9]:
enriched_train_features = enricher.transform(train_feature, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

90.39637% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=10290142-bf21-4f50-9667-d1625fd3c889
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done
90.36176% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=637fc363-674d-47f1-a645-c3b9ed57ccb5
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done


,date,store,item,f_weather_date_weather_pca_0_d7e0a1fc,f_events_date_week_sin1_847b5db1,f_events_date_week_cos1_f6a8c1fc,f_weather_date_weather_umap_48_b39cd0c4,f_weather_date_weather_umap_24_2e14c9a6,f_weather_date_weather_umap_33_89bb7578,f_events_date_year_cos1_9014a856,...,f_financial_date_dow_jones_7d_to_7d_1y_shift_61f71e90,f_events_date_italy_game_cnt_99570b80,f_weather_date_weather_umap_34_c3ef5b4f,f_economic_date_cbpol_pca_3_27450634,f_financial_date_nasdaq_c568533e,f_economic_date_cbpol_umap_1_7eb7a343,f_financial_date_silver_7d_to_7d_1y_shift_9cb6bdfc,f_weather_date_weather_umap_45_d474bf8d,f_economic_date_cbpol_umap_6_aa0352de,f_economic_date_cpi_umap_4_970cc061
0,2013-01-01,7,5,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,1.065267,0,5.664261,-0.323471,3019.51001,4.815701,1.072025,4.923654,1.367325,10.153208
1,2013-01-01,4,9,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,1.065267,0,5.664261,-0.323471,3019.51001,4.815701,1.072025,4.923654,1.367325,10.153208
2,2013-01-01,1,33,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,1.065267,0,5.664261,-0.323471,3019.51001,4.815701,1.072025,4.923654,1.367325,10.153208
3,2013-01-01,3,41,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,1.065267,0,5.664261,-0.323471,3019.51001,4.815701,1.072025,4.923654,1.367325,10.153208
4,2013-01-01,5,24,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,1.065267,0,5.664261,-0.323471,3019.51001,4.815701,1.072025,4.923654,1.367325,10.153208


In [11]:
model.fit(train_feature, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values,preds,"SMAPE")

[37.65141857448004]

In [14]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values,enriched_preds,"SMAPE")

[14.187522108368377]